# Oscilador armónico amortiguado

![](../../conTexto.png "DF, FCEyN, UBA")  
©2021 [Víctor A. Bettachini](mailto:bettachini@df.uba.ar)   

----   

## Amortiguación proporcional a la velocidad

![](shabana1996_fig5_6_sansF.png)

La dinámica del sistema de la figura estará caracterizada por  
- m : masa
- k : rigidez (_stiffness_) de un resorte que proveé una fuerza elástica según el modelo de Hooke, $\vec{F}_\mathrm{elástica} = - k x \hat{x}$
- c : coeficiente de la fuerza de amortiguación modelizada lineal con la velocidad, provista por un amortiguador en cuyo interior un pistón está sumergido en un fluido viscoso como el aceite, $\vec{F}_\mathrm{amortiguación} = - c \dot{x} \hat{x}$

In [1]:
# biblioteca de cálculo simbólico
import sympy as sym
import sympy.physics.mechanics as mech
mech.init_vprinting() # notación con puntos para derivadas temporales

In [28]:
t = sym.symbols('t') # tiempo
m, k, c = sym.symbols('m, k, c', positive = True) # características del sistema
x = mech.dynamicsymbols('x') # coordenada

In [29]:
r_m = x # posición de la masa 
unMedio = sym.Rational(1,2) # Rational: fracción de enteros, alternativamente podría haberse usado 0.5 
V = sym.Eq( sym.Symbol('V'), unMedio* k* r_m**2) # Energía potencial
V

Si no estuviera la fuerza de amortiguación, que es una fuerza no-conservativa, la dinámica estaría dictada por 

In [22]:
segundaLey = sym.Eq( m* x.diff(t,2), -V.rhs.diff(x) ) # diff: derivar, rhs: lado derecho de ecuación
segundaLey

La fuerzas no conservativas se limitan a la de amortiguación.

In [23]:
fNoConservativas = sym.Eq(sym.Symbol('F_{nc}'), - c* r_m.diff(t) )
fNoConservativas

Cualquier fuerza no-conservativa actuando sobre el sistema debe agregarse al lado derecho de esta ecuación que solo contempla las derivadas de un potencial, las conservativas.

In [27]:
segundaLeyNC = sym.Eq(segundaLey.lhs, segundaLey.rhs + fNoConservativas.rhs) # sumo las no-conservativas al lado derecho
segundaLeyNC

## Dinámica
Escribimos la anterior ecuación diferencial lineal como homogénea.

In [32]:
xDinámicaHomogénea =  sym.Eq(segundaLeyNC.lhs - segundaLeyNC.rhs, 0)
xDinámicaHomogénea

En ingeniería se suele normalizar por la masa, 

In [ ]:
x_EL_masa = (x_EL_homogenea/ m).expand()
sym.Eq(x_EL_masa, 0)

para asignar símbolos particulares a los coeficientes de $x$ y $\dot{x}$:
- frecuencia natural de oscilación del sistema : $\omega = \sqrt{\frac{k}{m} }$,
- razón (o tasa) de amortiguamiento: $\zeta = \frac{c}{C_c}$, donde $C_c = 2 \sqrt{k m} = 2 m \omega$ es el coeficiente de amortiguamiento crítico. Así $\frac{c}{m} = \frac{C_c \zeta}{m} = \frac{2 m \omega \zeta}{m} = 2 \omega \zeta$

In [ ]:
omega, zeta = sym.symbols('omega zeta', real = True)
x_EL_ing = x_EL_masa.subs([(k/m, omega**2), (c/m, 2* zeta* omega) ]).simplify()
sym.Eq((x_EL_ing).expand(), 0)

La solución en términos de exponenciales complejas

In [ ]:
x_EL_ing_sol = sym.dsolve(x_EL_ing)
x_EL_ing_sol.simplify()

es una combinación lineal de las dos posible soluciones, pesadas por los coeficientes $C_i$.

### Periodo de relajación $\tau$
Independientemente del comportamiento que presente $x(t)$ en función de $\zeta$, el término $\operatorname{e}^{- \omega \zeta t}$ se encarga de ir reduciendo la amplitud a medida que aumenta $t$.
De hecho cada vez que transcurre este **período de relajación** $\tau = \frac{1}{\omega \zeta}$ la amplitud decae en $\operatorname{e}^{-1}$.
Se considera que trancurrido $\sim 10 \tau$ la oscilación se considera extinta.

### Críticamente amortiguado, $\zeta = 1$
Si $\zeta = 1$ se logra el comportamiento más sencillo

In [ ]:
x_crit = x_EL_ing_sol.subs(zeta,1).simplify()
x_crit

que se trata de un acercamiento exponencial a la posición de equilibrio. 

Para visualizar estos comportamientos pueden determinarse los $C_i$ en función de los $x$ y $\dot{x}$ conocidos en un determinado $t$, las llamadas condiciones iniciales.

Si para $t_0 = 0$ conocemos $x_0 = x(t_0)$ y $\dot{x}_0 = \dot{x}(t_0)$

In [ ]:
x_0, dx_0 = sym.symbols('x_0 \dot{x}_0', real=True)
crit_condInic = [sym.Eq(x_0, x_crit.args[1].subs(t, 0) ), sym.Eq(dx_0, x_crit.args[1].diff(t).subs(t, 0) ) ]
crit_condInic

de las que pueden despejarse los $C_i$ 

In [ ]:
crit_condInic_sol = sym.solve(crit_condInic)
crit_condInic_sol

In [ ]:
x_crit_condInic = x_crit.subs(crit_condInic_sol[0] )
x_crit_condInic.simplify()

Si por ejemplo asignamos

In [ ]:
k = 100 # [N m-1]
m = 1 # [kg]
x_0_magnitud = 1 # [m]
dx_0_magnitud = -100 # [m s-1]

parametros ={
    x_0 : x_0_magnitud,
    dx_0 : dx_0_magnitud,
    omega : sym.sqrt(k/m)
}
x_crit_parametros = x_crit_condInic.subs(parametros)
x_crit_parametros

In [ ]:
sym.plot(x_crit_parametros.rhs, (t,0, 5), ylabel='x [m]', xlabel='t [s]') 

### Sobreamortiguado, $\zeta > 1$
Si el amortiguamiento es aún más pronunciado, e.g. $\zeta = 2$

In [ ]:
x_sobre = x_EL_ing_sol.subs(zeta, 2).simplify()
x_sobre

también habrá un acercamiento exponencial a $x=0$, pero puede haber violentas oscilaciones antes de alcanzar el punto de equilibrio.

In [ ]:
sobre_condInic = [sym.Eq(x_0, x_sobre.args[1].subs(t, 0) ), sym.Eq(dx_0, x_sobre.args[1].diff(t).subs(t, 0) ) ]
sobre_condInic_sol = sym.solve(sobre_condInic)
x_sobre_condInic = x_sobre.subs(sobre_condInic_sol[0] )
x_sobre_condInic.simplify()

Con las mismas condiciones iniciales

In [ ]:
x_sobre_parametros = x_sobre_condInic.subs(parametros)
sym.plot(x_sobre_parametros.rhs, (t,0, 5), ylabel='x [m]', xlabel='t [s]') 

### Subamortiguado, $0< \zeta < 1$
El caso más frecuente es cuando se busca que la amortiguación no sea brusca   

In [ ]:
x_sub = x_EL_ing_sol.subs(zeta, 0.2)
x_sub

Las exponenciales complejas con $i t$ pueden re-escribirse como funciones trigonométricas con igualdad de Euler $\mathrm{e}^{\pm i \theta}= \cos{\theta}+ i \sin{\theta}$.

$$
x= \left[ (C_1+ C_2) \cos{\omega t} + i(C_1- C_2) \sin{\omega t} \right] \operatorname{e}^{- \zeta \omega t}
$$
que presenta se observara una reducción de la amplitud de las oscilaciones en torno a la posición de equilibrio.


y escribir una solución de la dinámica en función de tales condiciones

In [ ]:
sub_condInic = [sym.Eq(x_0, x_sub.args[1].subs(t, 0) ), sym.Eq(dx_0, x_sub.args[1].diff(t).subs(t, 0) ) ]
sub_condInic_sol = sym.solve(sub_condInic)
x_sub_condInic = x_sub.subs(sub_condInic_sol[0] )
x_sub_condInic

In [ ]:
x_sub_parametros = x_sub_condInic.subs(parametros)
sym.plot(x_sub_parametros.rhs, (t,0, 5), ylabel='x [m]', xlabel='t [s]') 

### No amortiguado, $\zeta = 0$

In [ ]:
x_noAmortiguado = x_EL_ing_sol.subs(zeta, 0)
x_noAmortiguado

In [ ]:
noAmortiguado_condInic = [sym.Eq(x_0, x_noAmortiguado.args[1].subs(t, 0) ), sym.Eq(dx_0, x_noAmortiguado.args[1].diff(t).subs(t, 0) ) ]
noAmortiguado_condInic_sol = sym.solve(noAmortiguado_condInic)
x_noAmortiguado_condInic = x_noAmortiguado.subs(noAmortiguado_condInic_sol[0] )
x_noAmortiguado_condInic

In [ ]:
x_noAmortiguado_parametros = x_noAmortiguado_condInic.subs(parametros)
sym.plot(x_noAmortiguado_parametros.rhs, (t,0, 5), ylabel='x [m]', xlabel='t [s]') 